# 유저 정의 Sampler
- 나만의 샘플링 알고리즘을 실험할 수 있다
- 작업별로 알고리즘을 구현한다
- 다른 최적화 라이브러리를 래핑해서 Optuna 파이프라인에 통합한다

# Sampler 개요

- `Suggest API`(ex : `suggest_float()`)이 목적 함수 내에서 호출되면, `Distribution Object` (ex : `FloatDistribution`)이 내부적으로 생성된다. 
- Sampler는 이 Distribution에서 파라미터 값을 선택하여 `Suggest API`로 반환되고 목적함수에 의해 평가된다.

- 새로운 샘플러는 `BaseSampler`에서 시작한다. 
- 3개의 추상 메서드를 가진다.
    1. `infer_relative_serach_space()`
    2. `sample_relative()`
    3. `sample_independent()`  
  
  
  
  
- `Trial`이 시작되면 `infer_relative_search_space()`가 호출되어 상대 탐색 공간을 제공한다. 이후, `sample_relative()`가 간단한 상대 파라미터들을 샘플링한다. `sample_independent()`는 위의 관계성 탐색 공간에 상관 없이 독자적으로 파라미터를 샘플링한다.

- 자세한 내용은 [BaseSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.BaseSampler.html#optuna.samplers.BaseSampler)를 볼 것

# 예제 : SimulatedAnnealingSampler

In [5]:
import numpy as np
import optuna

class SimulatedAnnealingSampler(optuna.samplers.BaseSampler):
    def __init__(self, temperature = 100):
        self._rng = np.random.RandomState() # 난수생성기 객체를 만듦
        self._temperature = temperature
        self._current_trial = None
        
    def sample_relative(self, study, trial, search_space):
        if search_space == {}:
            return {}
        
        # Simulated Annealing
        # 1. Transition Probability 계산
        prev_trial = study.trials[-2]
        if self._current_trial is None or prev_trial.value <= self._current_trial.value:
            probability = 1.0
        else:
            probability = np.exp(
                (self._current_trial.value) - prev_trial.value
            ) / self._temperature
        self._temperature *= 0.9
        
        # 2. 이전 결과가 받아진다면 Current Value를 옮김
        if self._rng.uniform(0, 1) < probability: # .uniform : 균등분포
            self._current_trial = prev_trial
            
        # 3. 현재 지점의 근처에서 파라미터를 샘플링함
        # 여기서 샘플링된 파라미터는 다음 목적 함수의 실행에서 통과될 변수
        params = {}
        for param_name, param_distribution in search_space.items():
            if (
                # isinstance(a, b) : a가 b 타입이면 True, 아니면 False를 반환함
                not isinstance(param_distribution, optuna.distributions.FloatDistribution
                or (param_distribution.step is not None and param_distribution.step != 1)
                or param_distribution.log
            ):
                msg = (
                    "Only suggest_float() with 'step' 'None' or 1.0 and 'log' 'False' is supported"
                )
                raise NotImplementedError(msg)
            
            current_value = self._current_trial.params[param_name]
            width = (param_distribution.high - param_distribution.low) * 0.1
            neighbor_low = max(current_value - width, param_distribution.low)
            neighbor_high = min(current_value + width, param_distribution.high)
            params[param_name] = self._rng.uniform(neighbor_low, neighbor_high)
            
        return params
    
    def infer_relative_search_space(self, study, trial):
        return optuna.samplers.intersection_search_space(study)
    
    def sample_independent(self, study, trial, param_name, param_distribution):
        independent_sampler = optuna.samplers.RandomSampler()
        return independent_sampler.sample_independent(study, trial, param_name, param_distribution)
                

In [6]:
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    y = trial.suggest_float('y', -5, 5)
    return x**2 + y

sampler = SimulatedAnnealingSampler()
study = optuna.create_study(sampler = sampler)
study.optimize(objective, n_trials = 100)

best_trial = study.best_trial
print('Best Value : ', best_trial.value)
print('Parameters that achieve the best value : ', best_trial.params)

[I 2022-12-12 14:36:41,817] A new study created in memory with name: no-name-86646c65-c24d-4caa-91cf-d83ca6ac82c7
[I 2022-12-12 14:36:41,823] Trial 0 finished with value: 41.47600495972536 and parameters: {'x': -6.357852830110609, 'y': 1.0537123503798762}. Best is trial 0 with value: 41.47600495972536.
[I 2022-12-12 14:36:41,827] Trial 1 finished with value: 23.251029213087083 and parameters: {'x': -4.759151014721757, 'y': 0.601510832159952}. Best is trial 1 with value: 23.251029213087083.
[I 2022-12-12 14:36:41,830] Trial 2 finished with value: 28.930873344497133 and parameters: {'x': -5.276211926750573, 'y': 1.09246104851214}. Best is trial 1 with value: 23.251029213087083.
[I 2022-12-12 14:36:41,835] Trial 3 finished with value: 8.6350247618999 and parameters: {'x': -2.8318336516190556, 'y': 0.6157429314577845}. Best is trial 3 with value: 8.6350247618999.
[I 2022-12-12 14:36:41,838] Trial 4 finished with value: 2.678746148502399 and parameters: {'x': -1.452177377574519, 'y': 0.5699

Best Value :  -3.829766831274167
Parameters that achieve the best value :  {'x': -0.050705320636265316, 'y': -3.8323378608149934}


- 위 최적화에서 `x`, `y` 파라미터는 `SimulatedAnnealingSampler.sample_relative` 메서드를 사용해서 샘플링됨

엄밀히 말해, 1번째 Trial에서 `sample_independent` 메서드가 샘플링에 사용되었음. 왜냐면 `infer_relative_search_space()` 내의 `intersection_search_space()`는 완료된 trial이 없으면 탐색 공간을 추론하지 못하기 때문임